In [4]:
import requests
def check_model_status(model="bge-m3:latest", ollama_url="http://25.12.119.117:11435"):
    try:
        response = requests.get(f"{ollama_url}/api/tags")
        models = response.json()
        print(f"Available models: {models}")
        return model in [m['name'] for m in models.get('models', [])]
    except:
        return False
check_model_status()

Available models: {'models': [{'name': 'bge-m3:latest', 'model': 'bge-m3:latest', 'modified_at': '2025-07-24T07:15:58.120318652Z', 'size': 1157672605, 'digest': '7907646426070047a77226ac3e684fbbe8410524f7b4a74d02837e43f2146bab', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'bert', 'families': ['bert'], 'parameter_size': '566.70M', 'quantization_level': 'F16'}}, {'name': 'nomic-embed-text:v1.5', 'model': 'nomic-embed-text:v1.5', 'modified_at': '2025-07-21T14:32:08.767843002Z', 'size': 274302450, 'digest': '0a109f422b47e3a30ba2b10eca18548e944e8a23073ee3f3e947efcf3c45e59f', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'nomic-bert', 'families': ['nomic-bert'], 'parameter_size': '137M', 'quantization_level': 'F16'}}, {'name': 'llama2:latest', 'model': 'llama2:latest', 'modified_at': '2025-07-21T13:56:27.596918513Z', 'size': 3826793677, 'digest': '78e26419b4469263f75331927a00a0284ef6544c1975b826b15abdaef17bb962', 'details': {'parent_model': '', 'format': 'gguf',

True

In [1]:
from src import *
from datasets import load_dataset

dataset = load_dataset("pixparse/pdfa-eng-wds", streaming=True)
small_dataset = dataset["train"].take(100)
for doc in small_dataset:
    doc_name = doc["__key__"]
    pdf_doc = pdf_bytes_to_doc_stream(pdf_bytes=doc['pdf'], doc_name=f'{doc_name}.pdf')
    pdf_to_milvus(pdf=pdf_doc, embedding_model="BAAI/bge-m3", collection_name="pdfa_eng_wds_2")

Resolving data files:   0%|          | 0/1801 [00:00<?, ?it/s]

INFO:docling.datamodel.document:detected formats: [<InputFormat.PDF: 'pdf'>]
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.document_converter:Initializing pipeline for StandardPdfPipeline with options hash 1839ac7b8f395811156cc7d3e509f667
INFO:docling.models.factories.base_factory:Loading plugin 'docling_defaults'
INFO:docling.models.factories:Registered ocr engines: ['easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.models.factories.base_factory:Loading plugin 'docling_defaults'
INFO:docling.models.factories:Registered picture descriptions: ['vlm', 'api']
INFO:docling.pipeline.base_pipeline:Processing document 6368667.pdf
c:\Users\adler\Desktop\rag_proj\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argumen

=== 0 ===
chunk.text:
'Grade 8 -HP7 Physical Health Disease Prevention/ Health Promotion 2020 Virginia SOLs'
chunker.contextualize(chunk):
'Grade 8 -HP7 Physical Health Disease Prevention/ Health Promotion 2020 Virginia SOLs'

Grade 8 -HP7 Physical Health Disease Prevention/ Health Promotion 2020 Virginia SOLs
Sending request to http://25.12.119.117:11435/api/embeddings...
Response status: 200
{'embedding': []}
Error: No embedding found in response
Response content: {'embedding': []}
chunk metadata:None
=== 1 ===
chunk.text:
'Please see attached handout for a lesson submitted by a Virginia teacher'
chunker.contextualize(chunk):
'Description\nPlease see attached handout for a lesson submitted by a Virginia teacher'

Description
Please see attached handout for a lesson submitted by a Virginia teacher
Sending request to http://25.12.119.117:11435/api/embeddings...
Response status: 200
{'embedding': []}
Error: No embedding found in response
Response content: {'embedding': []}
chunk metadat

2025-07-28 16:31:07,899 [ERROR][handler]: RPC error: [insert_rows], <DataNotMatchException: (code=1, message=The Input data type is inconsistent with defined schema, {vector} field should be a float_vector, but got a {<class 'NoneType'>} instead.)>, <Time:{'RPC start': '2025-07-28 16:31:07.737425', 'RPC error': '2025-07-28 16:31:07.899202'}> (decorators.py:140)


DataNotMatchException: <DataNotMatchException: (code=1, message=The Input data type is inconsistent with defined schema, {vector} field should be a float_vector, but got a {<class 'NoneType'>} instead.)>

In [ ]:
# points = contextualize_embedd_and_metadata(chunks, embedder, embedding_model)
"sentence-transformers/all-MiniLM-L6-v2"

In [ ]:
question = "How to get started with docling?"
search_res = client.search(
    collection_name=collection_name,
    data=[embedd(question, embedder)],
    limit=3,
    search_params={"metric_type": "IP", "params": {}},  # Inner product distance
    output_fields=["text"],  # Return the text field
)
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))


In [ ]:
def semantic_search(query, limit=5):
    # הפיכת השאלה לוקטור
    query_vector = embedder.encode(query).tolist()
    
    # חיפוש
    results = client.search(
        collection_name=collection_name,
        query_vector=query_vector,
        limit=limit,
        with_payload=True
    )
    
    return results

# דוגמה לשימוש
query = "upsides of fisical activity"
results = semantic_search(query)

for result in results:
    print(f"ציון דמיון: {result.score:.3f}")
    print(f"טקסט: {result.payload['chunk_info']['']}")
    print("---")

In [ ]:
from qdrant_client.models import Filter, FieldCondition, MatchText

def hybrid_search(query, limit=5):
    query_vector = embedder.encode(query).tolist()
    
    # חיפוש עם פילטר טקסטואלי
    results = client.search(
        collection_name=collection_name,
        query_vector=query_vector,
        query_filter=Filter(
            should=[  # OR logic
                FieldCondition(
                    key="text_content",
                    match=MatchText(text=query)
                )
            ]
        ),
        limit=limit,
        with_payload=True
    )
    
    return results

query = "upsides of fisical activity"
results = hybrid_search(query)

for result in results:
    print(f"ציון דמיון: {result.score:.3f}")
    print(f"טקסט: {result.payload['chunk_info']['text_content']}")
    print("---")